Name: Taseng Mancheykhun, PRN: 240840141021

In [ ]:
%%writefile shared_countevenodd.cu
#include<stdio.h>
#define N 160

// 8 blocks, 20 threads each

__global__ void countevenodd(int *d_input, int *d_evenoutput, int *d_oddoutput, int n)
{
    __shared__ int sharedarray[20];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx<n)
    {
        sharedarray[threadIdx.x] = d_input[idx];
    }
    else
    {
        sharedarray[threadIdx.x] = 0;
    }

    __syncthreads();

    if (threadIdx.x == 0)
    {
      int counteven=0, countodd=0;
      for (int i=0; i<20; i++)
      {
           if (sharedarray[i]%2==0)
           {
                counteven+=1;
           }
           else
           {
                countodd+=1;
           }
      }

      d_evenoutput[blockIdx.x] = counteven;
      d_oddoutput[blockIdx.x] = countodd;
    }

}



int main()
{
    int h_input[N];
    int *d_input, *d_evenoutput, *d_oddoutput;
    int counteven=0, countodd=0;
    int h_evenoutput[8], h_oddoutput[8];

    for (int i=0;i<N;i++)
    {
        h_input[i]=i;
    }

    cudaMalloc(&d_input, N*sizeof(int));

    cudaMalloc(&d_evenoutput, 8*sizeof(int));

    cudaMalloc(&d_oddoutput, 8*sizeof(int));

    cudaMemcpy(d_input,h_input, N*sizeof(int), cudaMemcpyHostToDevice);

    countevenodd<<<8,20>>>(d_input, d_evenoutput, d_oddoutput, N);

    cudaMemcpy(h_evenoutput, d_evenoutput, 8*sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_oddoutput, d_oddoutput, 8*sizeof(int), cudaMemcpyDeviceToHost);


    printf("No. of EVEN numbers in each block \n");
    for (int i = 0; i<8; i++)
    {
      printf("%d ",h_evenoutput[i]);
    }
    printf("\n\n");


    printf("No. of ODD numbers in each block \n");
    for (int i = 0; i<8; i++)
    {
      printf("%d ",h_oddoutput[i]);
    }
    printf("\n\n");


    for (int i=0; i<8; i++)
    {
       counteven+=h_evenoutput[i];
       countodd+=h_oddoutput[i];
    }

    printf("Number of even numbers are: %d\n", counteven);
    printf("Number of odd numbers are: %d\n", countodd);


    cudaFree(d_input);
    cudaFree(d_evenoutput);
    cudaFree(d_oddoutput);

    return 0;
}

Overwriting shared_countevenodd.cu


In [ ]:
!nvcc -o exe shared_countevenodd.cu
!./exe

No. of EVEN numbers in each block 
10 10 10 10 10 10 10 10 

No. of ODD numbers in each block 
10 10 10 10 10 10 10 10 

Number of even numbers are: 80
Number of odd numbers are: 80
